# Imports + load cleaned data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, classification_report, confusion_matrix

In [2]:
df = pd.read_csv("../data/clean_data_eda.csv")
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,0


## Define target + feature groups

**Reasoning:** Logistic regression assumes linearity in log-odds; scaling helps optimization; one-hot avoids fake ordinal relationships for categories.

In [3]:
target = "Revenue"
y = df[target].astype(int)
X = df.drop(columns=[target])

# Identify columns
categorical_cols = ["Month", "VisitorType", "Weekend"]
# Treat these integer-coded identifiers as categorical for logistic regression
id_like_cols = ["OperatingSystems", "Browser", "Region", "TrafficType"]

# Numeric behavior features
numeric_cols = [c for c in X.columns if c not in categorical_cols + id_like_cols]

## Train/test split (stratified)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## We’ll evaluate:

- Baseline logistic (scaled numeric + one-hot categorical)

- Logistic with class_weight='balanced' (addresses imbalance explicitly)

- Logistic with log1p transform for skewed durations (addresses heavy right-skew)

# Model 1: Baseline pipeline

In [5]:
preprocess_base = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols + id_like_cols),
    ],
    remainder="drop"
)

logreg = LogisticRegression(max_iter=500, solver="lbfgs")

pipe_base = Pipeline([
    ("preprocess", preprocess_base),
    ("model", logreg)
])

# Model 2: Class-weighted logistic (imbalance-aware)

In [6]:
pipe_balanced = Pipeline([
    ("preprocess", preprocess_base),
    ("model", LogisticRegression(max_iter=500, solver="lbfgs", class_weight="balanced"))
])

# Model 3: Log-transform skewed numeric features

EDA shows strong right skew in durations; log transform can make linear separation in log-odds more plausible.

In [7]:
def log1p_df(X_arr):
    # ColumnTransformer passes numpy arrays; log1p safe for non-negative features
    return np.log1p(X_arr)

log_transformer = FunctionTransformer(log1p_df, feature_names_out="one-to-one")

preprocess_log = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("log1p", log_transformer), ("scaler", StandardScaler())]), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols + id_like_cols),
    ],
    remainder="drop"
)

pipe_log = Pipeline([
    ("preprocess", preprocess_log),
    ("model", LogisticRegression(max_iter=500, solver="lbfgs"))
])

# Evaluation function (metrics + time)

We’ll report:

- ROC-AUC (good for ranking, robust to imbalance)

- PR-AUC / Average Precision (more informative with imbalance)

- Training time

- classification report at a chosen threshold (default 0.5)

In [8]:
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

def eval_model(pipe, X_train, y_train, X_test, y_test, name="model"):
    t0 = time.time()
    pipe.fit(X_train, y_train)
    train_time = time.time() - t0

    proba = pipe.predict_proba(X_test)[:, 1]
    roc = roc_auc_score(y_test, proba)
    ap = average_precision_score(y_test, proba)

    print(f"\n=== {name} ===")
    print(f"Train time: {train_time:.3f} sec")
    print(f"ROC-AUC:    {roc:.4f}")
    print(f"PR-AUC(AP): {ap:.4f}")

    # Optional: threshold 0.5 report
    preds = (proba >= 0.5).astype(int)
    print("\nClassification report @0.5:")
    print(classification_report(y_test, preds, digits=4))

    return {"name": name, "train_time": train_time, "roc_auc": roc, "pr_auc": ap}

In [9]:
results = []
results.append(eval_model(pipe_base, X_train, y_train, X_test, y_test, "LogReg baseline"))
results.append(eval_model(pipe_balanced, X_train, y_train, X_test, y_test, "LogReg class_weight=balanced"))
results.append(eval_model(pipe_log, X_train, y_train, X_test, y_test, "LogReg log1p + scaled"))
pd.DataFrame(results).sort_values("roc_auc", ascending=False)


=== LogReg baseline ===
Train time: 0.059 sec
ROC-AUC:    0.8996
PR-AUC(AP): 0.6543

Classification report @0.5:
              precision    recall  f1-score   support

           0     0.8996    0.9747    0.9357      2059
           1     0.7524    0.4136    0.5338       382

    accuracy                         0.8869      2441
   macro avg     0.8260    0.6942    0.7347      2441
weighted avg     0.8766    0.8869    0.8728      2441


=== LogReg class_weight=balanced ===
Train time: 0.027 sec
ROC-AUC:    0.9097
PR-AUC(AP): 0.6627

Classification report @0.5:
              precision    recall  f1-score   support

           0     0.9576    0.8553    0.9035      2059
           1     0.5050    0.7958    0.6179       382

    accuracy                         0.8460      2441
   macro avg     0.7313    0.8255    0.7607      2441
weighted avg     0.8868    0.8460    0.8588      2441


=== LogReg log1p + scaled ===
Train time: 0.024 sec
ROC-AUC:    0.9217
PR-AUC(AP): 0.6952

Classificatio

,name,train_time,roc_auc,pr_auc
2,LogReg log1p + scaled,0.024270,0.921681,0.695160
1,LogReg class_weight=balanced,0.026568,0.909657,0.662708
0,LogReg baseline,0.058651,0.899574,0.654345


Among the logistic regression variants tested, the log-transformed model achieved the highest ROC-AUC (0.9217) and PR-AUC (0.6952), while maintaining a favorable balance between precision and recall. Given the heavy right-skew observed in duration-based predictors during exploratory analysis, the log1p transformation improves linear separability in the log-odds space. We therefore select the log-transformed logistic regression model as our final interpretable baseline.

In [11]:
# Fit the best-performing pipeline
best_pipe = pipe_log
best_pipe.fit(X_train, y_train)

# Get feature names after preprocessing
ohe = best_pipe.named_steps["preprocess"].named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(categorical_cols + id_like_cols)

num_feature_names = np.array(numeric_cols)
all_feature_names = np.concatenate([num_feature_names, cat_feature_names])

coef = best_pipe.named_steps["model"].coef_.ravel()
coef_df = pd.DataFrame({"feature": all_feature_names, "coef": coef}).sort_values("coef", ascending=False)

coef_df.head(15), coef_df.tail(15)

(               feature      coef
 8           PageValues  1.425969
 44          Browser_12  0.848150
 17           Month_Nov  0.788979
 45          Browser_13  0.598276
 62       TrafficType_8  0.540220
 31  OperatingSystems_7  0.520033
 64      TrafficType_10  0.411033
 13           Month_Jul  0.364679
 40           Browser_8  0.344032
 19           Month_Sep  0.264552
 61       TrafficType_7  0.256247
 67      TrafficType_14  0.253724
 10           Month_Aug  0.237652
 53            Region_8  0.237636
 73      TrafficType_20  0.235080,
                           feature      coef
 25             OperatingSystems_1 -0.292461
 23                  Weekend_False -0.350869
 15                      Month_Mar -0.359496
 50                       Region_5 -0.383292
 7                       ExitRates -0.402532
 54                       Region_9 -0.428000
 22  VisitorType_Returning_Visitor -0.440710
 16                      Month_May -0.471267
 39                      Browser_7 -0.535556
 11  

# Proper Threshold Tuning

Since the dataset is imbalanced (~15.6% positive), we should tune threshold based on F1 score (balanced precision/recall).

## Compute precision-recall curve

In [13]:
best_pipe = pipe_log
best_pipe.fit(X_train, y_train)
proba = best_pipe.predict_proba(X_test)[:, 1]

from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, proba)

# F1 calculation
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

# Find best threshold
best_idx = f1_scores.argmax()
best_threshold = thresholds[best_idx]

best_threshold, f1_scores[best_idx]

(np.float64(0.20987573478698007), np.float64(0.6774566473495256))

## Evaluate model at optimal threshold

In [15]:
from sklearn.metrics import classification_report

optimal_preds = (proba >= best_threshold).astype(int)

print("Optimal threshold:", round(best_threshold, 4))
print("\nClassification report @ optimal threshold:")
print(classification_report(y_test, optimal_preds, digits=4))

Optimal threshold: 0.2099

Classification report @ optimal threshold:
              precision    recall  f1-score   support

           0     0.9545    0.9077    0.9305      2059
           1     0.6066    0.7670    0.6775       382

    accuracy                         0.8857      2441
   macro avg     0.7806    0.8374    0.8040      2441
weighted avg     0.9001    0.8857    0.8909      2441



## Compare to default 0.5

In [16]:
default_preds = (proba >= 0.5).astype(int)

print("=== Default threshold 0.5 ===")
print(classification_report(y_test, default_preds, digits=4))

=== Default threshold 0.5 ===
              precision    recall  f1-score   support

           0     0.9275    0.9568    0.9419      2059
           1     0.7192    0.5969    0.6524       382

    accuracy                         0.9005      2441
   macro avg     0.8234    0.7768    0.7971      2441
weighted avg     0.8949    0.9005    0.8966      2441



# Extract and interpret the top 10 coefficients

In [17]:
best_pipe = pipe_log
best_pipe.fit(X_train, y_train)

# Get processed feature names
preprocess = best_pipe.named_steps["preprocess"]

# Numeric features (after log + scale)
num_feature_names = numeric_cols

# One-hot encoded categorical features
ohe = preprocess.named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(categorical_cols + id_like_cols)

# Combine feature names
all_feature_names = np.concatenate([num_feature_names, cat_feature_names])

# Extract coefficients
coef = best_pipe.named_steps["model"].coef_.ravel()

coef_df = pd.DataFrame({
    "feature": all_feature_names,
    "coef": coef
})

# Add odds ratios
coef_df["odds_ratio"] = np.exp(coef_df["coef"])

# Sort by magnitude (strongest effects)
coef_df_sorted = coef_df.reindex(
    coef_df["coef"].abs().sort_values(ascending=False).index
)

coef_df_sorted.head(10)

,feature,coef,odds_ratio
8,PageValues,1.425969,4.161888
12,Month_Feb,-0.949238,0.387036
44,Browser_12,0.848150,2.335322
35,Browser_3,-0.831441,0.435421
17,Month_Nov,0.788979,2.201148
66,TrafficType_13,-0.662648,0.515484
38,Browser_6,-0.638282,0.528199
45,Browser_13,0.598276,1.818980
68,TrafficType_15,-0.568432,0.566413
11,Month_Dec,-0.553007,0.575218


## Why We Use Odds Ratios in Logistic Regression

In logistic regression, the model is defined as

$$
\log\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 x_1 + \cdots + \beta_k x_k,
$$

where \( p = P(Y=1 \mid X) \).

Each coefficient \( \beta_j \) represents the change in the **log-odds** of the outcome for a one-unit increase in \( x_j \), holding other variables constant.  

Since log-odds are not directly interpretable, we exponentiate the coefficient:

$$
\text{Odds Ratio} = e^{\beta_j}
$$

The odds ratio tells us the **multiplicative change in the odds** of the event occurring.

- If \( e^{\beta_j} > 1 \): odds increase  
- If \( e^{\beta_j} < 1 \): odds decrease  
- If \( e^{\beta_j} = 1 \): no effect  

Thus, odds ratios provide a clear and interpretable measure of effect size.

## Interpretation of Selected Results

### 1. PageValues  
$$
\beta = 1.426, \quad OR = 4.16
$$

For each one-unit increase in `PageValues`, the odds of purchase are multiplied by **4.16**, holding other variables constant.  
This is the strongest positive predictor, indicating that higher page value is strongly associated with conversion.

---

### 2. Month_Feb  
$$
\beta = -0.949, \quad OR = 0.387
$$

Compared to the reference month, sessions in February have odds of purchase that are **0.387 times as large**, meaning approximately **61% lower odds** of conversion.

---

### 3. Browser_12  
$$
\beta = 0.848, \quad OR = 2.34
$$

Users on Browser 12 have **2.34 times higher odds** of purchase relative to the baseline browser.

---

### 4. Browser_3  
$$
\beta = -0.831, \quad OR = 0.435
$$

Users on Browser 3 have odds of purchase that are approximately **56.5% lower** than the reference browser.

---

### 5. Month_Nov  
$$
\beta = 0.789, \quad OR = 2.20
$$

Sessions in November have **2.20 times higher odds** of purchase compared to the baseline month, likely reflecting seasonal shopping behavior.

---

### 6. Other Traffic Types and Browsers

Several features such as `TrafficType_13` (OR = 0.52) and `TrafficType_15` (OR = 0.57) have odds ratios below 1, indicating reduced likelihood of conversion compared to their reference categories.

## Summary

The largest positive effect comes from **PageValues**, while seasonal indicators (November positive, February negative) and browser/traffic type variables also meaningfully impact purchase probability.

Using odds ratios allows us to interpret effects multiplicatively on the odds scale, which is substantially more intuitive than interpreting raw log-odds coefficients.

# Final Summary

## 1. Modeling Objective

The goal of this analysis is to predict purchase behavior (`Revenue`) using session-level behavioral, temporal, and technical features. Since the response variable is binary, logistic regression is an appropriate modeling choice because it directly models:

$$
\log\left(\frac{p}{1-p}\right) = \beta_0 + \beta^T X
$$

where \( p = P(\text{Revenue}=1 \mid X) \).

Logistic regression provides:
- Probabilistic outputs
- Interpretable coefficients
- Strong baseline performance
- Fast training time

## 2. Preprocessing Justification

### (a) Scaling Numeric Features

Logistic regression assumes linearity in the log-odds. Scaling ensures:
- Faster convergence for the optimizer
- Comparable coefficient magnitudes
- Numerical stability

### (b) One-Hot Encoding Categorical Features

Integer-coded variables such as `Browser`, `Region`, and `TrafficType` were treated as categorical to avoid imposing artificial ordinal structure.

### (c) Log-Transformation of Skewed Variables

EDA revealed strong right-skew in duration-based features.  
We applied:

$$
x \rightarrow \log(1 + x)
$$

This transformation:
- Reduces skewness
- Improves linear separability in log-odds space
- Enhances model performance

## 3. Model Comparison and Selection

We evaluated three variants:

| Model | ROC-AUC | PR-AUC |
|-------|---------|--------|
| Baseline | 0.8996 | 0.6543 |
| Class-weighted | 0.9097 | 0.6627 |
| Log1p + Scaled | **0.9217** | **0.6952** |

The log-transformed model achieved the highest ROC-AUC and PR-AUC, indicating superior ranking ability and better performance under class imbalance (~15.6% positives).

We therefore select the **log1p + scaled logistic regression** as our final interpretable baseline.


## 4. Threshold Tuning

Because the dataset is imbalanced, the default threshold (0.5) is not necessarily optimal.

We tuned the classification threshold using the F1 score and found:

$$
\text{Optimal threshold} \approx 0.21
$$

At this threshold:

- Recall for purchasers increased substantially
- F1 score improved to 0.677
- The model better balances precision and recall

This is important in conversion prediction, where identifying potential purchasers is often more valuable than maximizing overall accuracy.

## 5. Interpretation of Key Predictors (Odds Ratios)

Logistic coefficients are interpreted through odds ratios:

$$
\text{Odds Ratio} = e^{\beta}
$$

### Strongest Positive Effects

- **PageValues (OR = 4.16)**  
  A one-unit increase multiplies the odds of purchase by over 4×.  
  This is the most influential predictor.

- **Month_Nov (OR = 2.20)**  
  Sessions in November have more than double the purchase odds, reflecting seasonal effects.

- **Browser_12 (OR = 2.34)**  
  Certain browser types are strongly associated with higher conversion likelihood.

### Strongest Negative Effects

- **Month_Feb (OR = 0.39)**  
  Approximately 61% lower odds relative to baseline month.

- **Browser_3 (OR = 0.44)**  
  Indicates significantly lower purchase likelihood.

These effects are multiplicative on the odds scale and therefore naturally interpretable.

## 6. Overall Conclusion

The final log-transformed logistic regression model:

- Achieves strong discrimination (ROC-AUC = 0.9217)
- Handles imbalance effectively through threshold tuning
- Maintains interpretability via odds ratios
- Trains extremely quickly (< 0.03 seconds)

This model provides a robust, interpretable baseline for predicting online shopping conversion and establishes a strong foundation for comparison with more complex models (e.g., tree-based methods).